In [63]:
# 处理照片Exif信息
import exifread
import os

def getExif(filename):
    fd = open(filename,'rb')
    tags = exifread.process_file(fd)
    fd.close()
    return(tags)

filename = "test.jpg"
tags=getExif(filename)

for tag in tags.keys():
    if tag in ('EXIF DateTimeOriginal', 'Image Make'):
        print("Key: %s, value: %s" % (tag, tags[tag]))

Key: Image Make, value: HUAWEI
Key: EXIF DateTimeOriginal, value: 2017:11:14 07:22:39


In [1]:
import os

dir=os.listdir("C:\\Mydata")
for d in dir:
    print(d)

desktop.ini
Miniconda3
Tools
坚果云
